In [ ]:
#Loading in Packages and Data

#Importing Packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
from matplotlib.colors import Normalize
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter
import matplotlib.colors as mcolors
import matplotlib.gridspec as gridspec
import xarray as xr
import os; import time
import pickle
import h5py
###############################################################
def coefs(coefficients,degree):
    coef=coefficients
    coefs=""
    for n in range(degree, -1, -1):
        string=f"({coefficients[len(coef)-(n+1)]:.1e})"
        coefs+=string + f"x^{n}"
        if n != 0:
            coefs+=" + "
    return coefs
###############################################################

#Importing Model Data
check=False
dir='/mnt/lustre/koa/koastore/torri_group/air_directory/DCI-Project/'
data=xr.open_dataset(dir+'cm1r20.3/run/cm1out_test7tundra-7_062217.nc') #***
true_time=data['time']
parcel=xr.open_dataset(dir+'cm1r20.3/run/cm1out_pdata_test5tundra-7_062217.nc') #***
times=data['time'].values/(1e9 * 60); times=times.astype(float);
# parcel=parcel.isel(time=times.astype(int)) 

#Restricts the timesteps of the data from timesteps0 to 140
data=data.isel(time=np.arange(0,140+1))
parcel=parcel.isel(time=np.arange(0,140+1))

def grid_location(x,y,z):
    xf=data['xf'].values*1000; which_xh=np.searchsorted(xf,x)-1; which_xh=np.where(which_xh == -1, 0, which_xh) #finds which x layer parcel in
    yf=data['yf'].values*1000; which_yh=np.searchsorted(yf,y)-1; which_yh=np.where(which_yh == -1, 0, which_yh) #finds which y layer parcel in
    zf=data['zf'].values*1000; which_zh=np.searchsorted(zf,z)-1; which_zh=np.where(which_zh == -1, 0, which_zh) #finds which z layer parcel in   
    return which_zh,which_yh,which_xh
    
# def grid_location(x,y,z): #faster
#     #finding xf and yf
#     ybins=data['yf'].values*1000; dy=ybins[1]-ybins[0] #1000
#     xbins=data['xf'].values*1000; dx=xbins[1]-xbins[0] #1000
#     dy=np.round(dy);dx=np.round(dx)

#     #digitizing
#     zf=data['zf'].values*1000; which_zh=np.searchsorted(zf,z)-1; which_zh=np.where(which_zh == -1, 0, which_zh) #finds which z layer parcel in 
#     if which_zh.ndim==0:
#         which_zh=np.array([which_zh])
#     which_yh=np.floor(y/dy).astype(int)+np.where(data['yf']==0)[0]
#     which_xh=np.floor(x/dx).astype(int)+np.where(data['xf']==0)[0]

#     #fixing boundaries
#     which_zh[np.where(which_zh==len(data['zh']))]-=1
#     which_yh[np.where(which_yh==len(data['yh']))]-=1
#     which_xh[np.where(which_xh==len(data['xh']))]-=1
#     return which_zh,which_yh,which_xh

In [ ]:
##########################################################################################
#Flag Arrays

In [ ]:
# save parcel_flag_array storing parcel id at location in eulerian grid at all times

yes_run=False
yes_run=True #uncomment if running

if yes_run==True: 

    if 'emptylike' not in globals():
        print('making empty array')
        flag_array = np.empty_like(data['qc']) #uses a variable that is (zh,yh,xh)
        empty_array = flag_array.copy()[0]

        output_file = dir+f'tracking_algorithms/plots/parcel_flag_array.h5' 
        with h5py.File(output_file, 'w') as f:
            f.create_dataset('flag_array', data=flag_array, compression="gzip")
        del flag_array
        emptylike=True
        print('done')
          
    ############################################################
    for t in np.arange(len(data['time'])):
    # for t in [33,34]: #TESTING
        print(f'current timestep {t}')
        parcel_flag=empty_array.copy()

        indexes=parcel['xh'].values 
        xs=parcel['x'].isel(time=t).values
        ys=parcel['y'].isel(time=t).values
        zs=parcel['z'].isel(time=t).values

        position=grid_location(xs,ys,zs)

        #OLD
        # parcel_flag[position]=indexes
        #NEW
        position_tuples = list(zip(position[0], position[1], position[2]))
        one,two=np.unique(position_tuples,return_counts=True,axis=0)
        parcel_flag[tuple(zip(*one))] += two

        with h5py.File(output_file, 'a') as f:
            f['flag_array'][t] = parcel_flag
        del parcel_flag 

In [ ]:
# OUT_NZ

# out=xr.open_dataset(dir+'tracking_algorithms/trackout/parcel_tracking4tundra-7_062217.nc')['out_arr'].values;out=out.astype(object);out[:, [0,1,2,4,5]] = out[:, [0,1,2,4,5]].astype(int) #***
# save=xr.open_dataset(dir+'tracking_algorithms/trackout/parcel_tracking4tundra-7_062217.nc')['save_arr'].values;save=save.astype(object);save[:, [0,1,2,4,5]] = save[:, [0,1,2,4,5]].astype(int) #***

out=xr.open_dataset(dir+'tracking_algorithms/trackout/parcel_tracking_everywhere.nc')['out_arr'].values;out=out.astype(object);out[:, [0,1,2,4,5]] = out[:, [0,1,2,4,5]].astype(int) #***
save=xr.open_dataset(dir+'tracking_algorithms/trackout/parcel_tracking_everywhere.nc')['save_arr'].values;save=save.astype(object);save[:, [0,1,2,4,5]] = save[:, [0,1,2,4,5]].astype(int) #***

out_nz=out[~np.all(out == 0, axis=1)];#print('list of first 10 SBZ parcels'); print(out_nz[:15])
save_nz=save[~np.all(save == 0, axis=1)];save_nz=save_nz[np.where(np.unique(save_nz[1:-1,0]))];#print('list of first 10 ignored parcels');print(save_nz[:5])

###############################################################################
#remove duplicates
lst=[]
unique_values, counts = np.unique(out_nz[:,0], return_counts=True); duplicates = unique_values[counts > 1]
for elem in duplicates:
    idx = np.where(out_nz[:,0] == elem)[0] 
    extras=idx[np.where(out_nz[idx,5]!=np.nanmin(out_nz[idx,5]))]
    lst.extend([x for x in extras])
mask=np.ones(len(out_nz), dtype=bool); mask[lst] = False
out_nz=out_nz[mask]; 
placeholder=out_nz.copy(); run=True
###############################################################################

# out_nz=out[~np.all(out == 0, axis=1)];print('list of first 10 SBZ parcels'); print(out_nz[:15])
# save_nz=save[~np.all(save == 0, axis=1)];save_nz=save_nz[np.where(np.unique(save_nz[1:-1,0]))];print('list of first 10 ignored parcels');print(save_nz[:5])
# print(f'there are a total of {len(out_nz)} SBZ parcels and {len(save_nz)} forgotten parcels')

#search for deep convective parcels within lagrangian tracking output     
##############################################################
def threshold(zthresh):
    out_nz=placeholder.copy()
    
    deep_out_ind=[]; extendrange=[]
    times=data['time'].values/(1e9 * 60); times=times.astype(float);
    for ind in range(len(out_nz)): 
        #searchs if next most local max goes above zthresh
        nummins=120; numsteps=int(nummins/times[1])
        aboverange=np.arange(out_nz[ind,5],out_nz[ind,5]+numsteps,1) #range of times between current time and numsteps later
        aboverange=aboverange[aboverange<len(data['time'])] #caps out at max time
        above=parcel['z'].isel(xh=out_nz[ind,0],time=aboverange).values/1000
    
        dt=1
        #takes dabove/dt
        f=above
        ddx = (
                f[1:  ]
                -
                f[0:-1]
            ) / (
            2 * dt
        )
        signs = np.sign(ddx)
        signs_diff=np.diff(signs)
        local_maxes=np.where((signs_diff != 0) & (signs_diff < 0))[0]+1 #make sure +1 is here
        if len(local_maxes)==0:
            local_maxes=[0]
        
        if np.any(above[local_maxes[0]]>zthresh):
            extendrange.append(local_maxes[0]) #save to extend xlim of plot later
            deep_out_ind.append(ind)
    
    out_nz=out_nz[deep_out_ind,:]
    # print(f'> {zthresh} km. {len(out_nz)} leftover parcels')
    return out_nz, extendrange
    # print(out_nz)
##############################################################

# convectivelevel=4 #4km
convectivelevel=6 #6km
# convectivelevel=8 #8km
# convectivelevel=10 #8km
[out_nz,extendrange]=threshold(convectivelevel)

print('list of first 10 SBZ parcels'); print(out_nz[:15])
print(f'there are a total of {len(out_nz)} SBZ parcels and {len(save_nz)} forgotten parcels')


#DEEP CONVECTIVE VERSION (run only if using Deep Convective)
#Setting up run for deep convective parcels
# np.arange(Np)==>out_nz[:,0]
# np.arange(Nt)==>out_nz[:,4]
col4 = out_nz[:, 4].astype(int)
# col5 = out_nz[:, 5].astype(int)
col5 = out_nz[:, 5].astype(int) + 2 #add another 10 minutes to each trajectory
ranges = [np.arange(start, end + 1) for start, end in zip(col4, col5)]
indexes = out_nz[:,0]
indexranges = [np.full_like(r, fill_value=index) for r, index in zip(ranges, indexes)]
totalelements=sum(len(arr) for arr in ranges)
print(f'running for total of {totalelements} elements')

x=parcel['x'];y=parcel['y'];z=parcel['z']
which_z,which_y,which_x=grid_location(x,y,z)

#Makes np.where style mask for deep parcel locations

deep_t,deep_z,deep_y,deep_x=[],[],[],[]
for R in np.arange(len(ranges)):
    tup=(ranges[R],indexranges[R])
    deep_t.extend(ranges[R])
    deep_z.extend(which_z[tup])
    deep_y.extend(which_y[tup])
    deep_x.extend(which_x[tup])
deep_t=np.array(deep_t);deep_z=np.array(deep_z);deep_y=np.array(deep_y);deep_x=np.array(deep_x)
print('done')

In [ ]:
# save DEEP_parcel_flag_array storing parcel id at location in eulerian grid at all times
# all locations marked 1 regardless of how many lagrangian parcels located there.

yes_run=False
yes_run=True #uncomment if running

if yes_run==True: 

    if 'emptylike' not in globals():
        print('loading neccessary variables')
        variable='w'; w_data=data[variable] #get w data
        w_data=w_data.interp(zf=data['zh']).data #interpolation w data z coordinate from zh to zf
        variable='qc'; qc_data=data[variable].data # get qc data
        variable='qi'; qi_data=data[variable].data # get qc data
        qc_plus_qi=qc_data+qi_data
        print('done loading')
        emptylike=True
    
    for type in ['all','cloudy','updraft','cloudyupdraft']:
        print(f"currently on type {type}")
    

        print('making empty array')
        flag_array = np.empty_like(data['qc']) #uses a variable that is (zh,yh,xh)     
        position=(deep_t,deep_z,deep_y,deep_x)

        w_thresh=1
        qcqi_thresh=1e-6
        if type=='all':
            #thresholding flag_array
            flag_array[position]=1 
            pass
        elif type=='cloudy':
            mask1 = position
            mask2 = np.where(qc_plus_qi >= qcqi_thresh) #*
        elif type=='updraft':
            mask1 = position
            mask2 = np.where(w_data >= w_thresh)
        elif type=='cloudyupdraft':
            mask1 = position
            mask2 = np.where((w_data >= w_thresh) & (qc_plus_qi >= qcqi_thresh))


        if type!='all': #only happens when applying threshold
            mask1_set = set(zip(*mask1))  
            mask2_set = set(zip(*mask2)) 
            shared_positions = mask1_set.intersection(mask2_set)
            position_array = np.array(list(shared_positions))
            position2 = (position_array[:, 0], position_array[:, 1], position_array[:, 2], position_array[:, 3])
            #thresholding flag_array
            flag_array[position2]=1     

        
        if type=='all':
            output_file = dir+f'tracking_algorithms/plots/DEEP_all_flag_array.h5' 
        elif type=='cloudy':
            output_file = dir+f'tracking_algorithms/plots/DEEP_cloudy_flag_array_wthresh{w_thresh}.h5' 
        elif type=='updraft':
            output_file = dir+f'tracking_algorithms/plots/DEEP_updraft_flag_array_wthresh{w_thresh}.h5' 
        elif type=='cloudyupdraft':
            output_file = dir+f'tracking_algorithms/plots/DEEP_cloudyupdraft_flag_array_wthresh{w_thresh}.h5' 
        with h5py.File(output_file, 'w') as f:
            f.create_dataset('flag_array', data=flag_array, compression="gzip")
        del flag_array
        
        print('done')
          

In [ ]:
#created flag array with locations of all parcels in eulerian frame
#only save locations where the parcel is in a cloudy region

# save flag_array updraft, cloudy, or cloudy_updraft
yes_run=False 
yes_run=True #uncomment if running

def make_flag_array(type):
    global w_thresh
    w_thresh=1
    qc_thresh=1e-6
    qcqi_thresh=1e-6
    
    if yes_run==True: 
        
        if 'emptylike' not in globals():
            print('making empty array')
            flag_array = np.empty_like(data['qc'])
            empty_array = flag_array.copy()[0]
            if type=='cloudy':
                output_file = dir+f'tracking_algorithms/plots/cloudy_flag_array_wthresh{w_thresh}.h5' 
            elif type=='updraft':
                output_file = dir+f'tracking_algorithms/plots/updraft_flag_array_wthresh{w_thresh}.h5' 
            elif type=='cloudyupdraft':
                output_file = dir+f'tracking_algorithms/plots/cloudyupdraft_flag_array_wthresh{w_thresh}.h5' 
            else:
                raise ValueError('Invalid type: {}. Please use "cloudy", "updraft", or "cloudyupdraft".'.format(type))
        
            with h5py.File(output_file, 'w') as f:
                f.create_dataset('flag_array', data=flag_array, compression="gzip")
            del flag_array
            emptylike=True
            print('done')
        
        ############################################################
        for t in np.arange(len(data['time'])):
        # for t in [33,34]: #TESTING
            print(f'current timestep {t}')
            initial_flag=empty_array.copy()
            parcel_flag=empty_array.copy()
            
            xs=parcel['x'].isel(time=t).values
            ys=parcel['y'].isel(time=t).values
            zs=parcel['z'].isel(time=t).values

            position=grid_location(xs,ys,zs)
            initial_flag[position]=1
            
            for which_zh in np.arange(len(data['zh'])):
                w_plane=data['w'].isel(time=t).interp(zf=data['zh']).isel(zh=which_zh).values #xy plane of w
                qc_plane=data['qc'].isel(time=t,zh=which_zh).values #xy plane of qc
                qi_plane=data['qi'].isel(time=t,zh=which_zh).values #xy plane of qi
                qc_plus_qi_plane=qc_plane+qi_plane
                
                if type=='cloudy':
                    cloudy_where=np.where((initial_flag[which_zh] == 1) & (qc_plane>=qc_thresh)) 
                    parcel_flag[which_zh][cloudy_where]=1
                elif type=='updraft':
                    updraft_where=np.where((initial_flag[which_zh] == 1) & (w_plane>=w_thresh))
                    parcel_flag[which_zh][updraft_where]=1
                elif type=='cloudyupdraft':
                    cloudyupdraft_where=np.where((initial_flag[which_zh] == 1) & (w_plane>=w_thresh) & (qc_plus_qi_plane>=qcqi_thresh))
                    parcel_flag[which_zh][cloudyupdraft_where]=1
                         
            # flag_array[t]=parcel_flag #OLD
            with h5py.File(output_file, 'a') as f:
                f['flag_array'][t] = parcel_flag
            del parcel_flag
            ############################################################

make_flag_array(type='cloudy')
make_flag_array(type='updraft')
make_flag_array(type='cloudyupdraft')

In [ ]:
##########################################################################################
#Functions

In [ ]:
def shape_search_2d(array): #2-D Shape Searching Algorithm (based on breadth-first search) 
    array=array.copy()
    final_array=np.zeros_like(array)
    #Functions 
    def next_most_1_2d(array): #finds next most 1
        if len(np.where(array==1)[0])!=0:
            result=tuple(arr[0] for arr in np.where(array==1))
            return result
        else:
            return (-1,-1)
    ######################################################################   
    def find_neighbor_index_2d(array,container): #find the indexes of all neighbors
                    
        #neighbors that are 0 or exist in visit or queue are not appended to neighbors
        tup,neighbors=[],[]
        container=[tuple(sub) for sub in container]; 
        nx,ny=array.shape[1]-1,array.shape[0]-1
        
        for index in container:
            if index==(-1,-1):
                pass
            else: 
                ######################################################################   
                #code for inside #all possible neighbors
                tup=[[index[0]-1,index[1]-1],[index[0]-1,index[1]],[index[0]-1,index[1]+1], 
                    [index[0],index[1]-1],[index[0],index[1]+1],
                    [index[0]+1,index[1]-1],[index[0]+1,index[1]],[index[0]+1,index[1]+1]] 
                #code for edges and corners 
                tup=[[np.mod(sublist[0],ny+1),sublist[1]] for sublist in tup] #use if y boundaries are periodic
                # tup=[sublist[0],sublist[1],[np.mod(sublist[1],nx+1)] for sublist in tup] #use if x boundaries are periodic
                tup=[sublist for sublist in tup if 0 <= sublist[1] <= nx] #use if x not periodic 
                if tup: [neighbors.append(sub) for sub in tup]
                ######################################################################             
            if neighbors:  #only unique elements that are not visited
                neighbors=np.unique(neighbors,axis=0).tolist()
                neighbors=[tuple(sub) for sub in neighbors if array[tuple(sub)] != 0 and tuple(sub) not in container]

        if not neighbors:
            neighbors=np.full((0,2),0,dtype=int)    
        else: neighbors=np.array(neighbors)
        return neighbors
    ###################################################################### 
    
    #The Algorithm
    n,k=0,0 #n is current shape, k is number of iterations on current shape 
    while True:
        n+=1;
        # if np.mod(n,5)==0:
        #     print("Current shape number: " + str(n))
        visit=np.full((0,2),0,dtype=int) #creates visit variable for current shapes
        queue=np.full((0,2),0,dtype=int) #creates queue variable for possible neighbors

        ######################################################################
        #finds the first 1 in the array, append to visit, and zero out
        index=next_most_1_2d(array);
        visit=np.concatenate([visit,[index]]);array[index]=0;
        
        #finds neighbors of first index and appends to queue
        neighbors=find_neighbor_index_2d(array,visit) 
        queue=np.concatenate([queue,neighbors]) 

        #for rest of individual shape
        while queue.size!=0: 
            k+=1
            #add all queued indexes to visit and find unique,nonzero,and nonvisited neighbor
            visit=np.concatenate([visit,queue]); neighbors=find_neighbor_index_2d(array,queue) 
            #zero out queued in array and empty out queue
            array[queue[:, 0], queue[:, 1]] = 0; queue=np.full((0,2),0,dtype=int) 
            #append found neighbor to queue
            queue=np.concatenate([queue,neighbors])
                
            #failsafes to end while loop
            if queue.size==0: 
                break #breaks single loop for current shape if no more neighbor
            if k>=array.shape[0]*array.shape[1]: #k failsafe, set to max volume of array
                break    
        #sets output array's value to shape number
        if index!=(-1,-1):
            final_array[visit[:, 0], visit[:, 1]] = n #sets final array location to shape number
        ######################################################################  
        
        #failsafes to end while loop
        if index==(-1,-1):
            # print("no more shapes are left")
            # print(f"total of {n-1} shapes were found")
            break  #break loop if no more 1s
        if n>=array.shape[0]*array.shape[1]:  #n failsafe, set to max volume of array
            print("failsafe reached")
            break  #break loop if no more 1s
    return final_array
##########################################################################################################################################  

In [ ]:
##########################################################################################
#Loading Deep convective Parcels

In [ ]:
#################################################
# Average Cloud Properties

In [ ]:
#import parcel_flag_array
w_thresh=1
import h5py
# input_file = dir+'tracking_algorithms/plots/parcel_flag_array.h5' 
# with h5py.File(input_file, 'r') as f:
#     parcel_flag_array = np.array(f['flag_array'])

#import cloudy, updraft, or updraft_cloudy flag_array
input_file = dir+f'tracking_algorithms/plots/cloudy_flag_array.h5' 
with h5py.File(input_file, 'r') as f:
    cloudy_flag_array = np.array(f['flag_array'])

input_file = dir+f'tracking_algorithms/plots/updraft_flag_array_wthresh{w_thresh}.h5' 
with h5py.File(input_file, 'r') as f:
    updraft_flag_array = np.array(f['flag_array'])

input_file = dir+f'tracking_algorithms/plots/cloudyupdraft_flag_array_wthresh{w_thresh}.h5' 
with h5py.File(input_file, 'r') as f:
    cloudyupdraft_flag_array = np.array(f['flag_array'])

zerowthresh=False

In [ ]:
# #TESTING TESTING ***
# test=cloudy_flag_array[33]==updraft_flag_array[33]
# test=cloudy_flag_array==updraft_flag_array
# percent=np.where(test==False)[0].shape[0]*100/(test.shape[0]*test.shape[1]*test.shape[2]*test.shape[3])
# print(f'{percent}%')

# test=cloudyupdraft_flag_array[33]==updraft_flag_array[33]
# test=cloudyupdraft_flag_array==updraft_flag_array
# percent=np.where(test==False)[0].shape[0]*100/(test.shape[0]*test.shape[1]*test.shape[2]*test.shape[3])
# print(f'{percent}%')

# test=cloudyupdraft_flag_array[33]==cloudy_flag_array[33]
# test=cloudyupdraft_flag_array==cloudy_flag_array
# percent=np.where(test==False)[0].shape[0]*100/(test.shape[0]*test.shape[1]*test.shape[2]*test.shape[3])
# print(f'{percent}%')

In [ ]:
#Job Array

num_jobs=30 #how many total jobs are being run? i.e. array=1-100 ==> num_jobs=100 #***

job_id = int(os.environ.get('SLURM_ARRAY_TASK_ID', 0)) #this is the current SBATCH job id
if job_id==0: job_id=1
num_parcels=len(data['time']) #total number of parcels
job_range = num_parcels//num_jobs #number of parcels per job 

# Calculate start and end based on job_id
start_job = (job_id - 1) * job_range
end_job = start_job + job_range
if job_id==num_jobs: end_job=num_parcels-1
print(f'running for timesteps {start_job}-{end_job-1}')

data=data.isel(time=slice(start_job,end_job))
# index_adjust = int(data['time'][0].values/1e9/60)
parcel=parcel.isel(time=slice(start_job,end_job))

parcel_flag_array=parcel_flag_array[slice(start_job,end_job)]
cloudy_flag_array=cloudy_flag_array[slice(start_job,end_job)]
updraft_flag_array=updraft_flag_array[slice(start_job,end_job)]
cloudyupdraft_flag_array=cloudyupdraft_flag_array[slice(start_job,end_job)]

 

In [ ]:
#Collected data of averaged BFS clouds for all clouds, updrafts, and cloudy updrafts (need to change the code up to make it a 2d histogram)
types=['cloudy','updraft','cloudyupdraft']

for type in types:
    print(f'current type: {type}')
    # if type=='cloudyupdraft': break #TESTING
    
    if type=='cloudy':
        flag_array=cloudy_flag_array.copy()
    elif type=='updraft':
        flag_array=updraft_flag_array.copy()
    elif type=='cloudyupdraft':
        flag_array=cloudyupdraft_flag_array.copy()
    
    def all_clouds_single_layer(t,which_zh,cloudy_bfs,vars):
        #appends the means of each individual cloud into a list
        max_ind=int(np.nanmax(cloudy_bfs))
        for ind in np.arange(1,max_ind+1):
            position=np.where(cloudy_bfs==ind)
    
            if np.any(position)==True:
                for variable in vars:
                    if variable=='w':
                        var_cloud_data=data[variable].isel(time=t).interp(zf=data['zh']).isel(zh=which_zh)[position].values
                    else:
                        var_cloud_data=data[variable].isel(time=t).isel(zh=which_zh)[position].values
                    if variable=='qv' or variable=='qc':
                        var_cloud_data*=1000
                        
                    var_cloud_data=np.mean(var_cloud_data)
                    globals()[f"{variable}_values"].append(var_cloud_data)
                    if variable=='w': #only perform once
                        zlevels.append(which_zh)
        return (globals()[f"{variable}_values"] for variable in vars), zlevels
    
    def average_clouds(t,vars):
        global zlevels
        parcel_flag=flag_array[t]
        for which_zh in np.arange(len(data['zh'])):
            if np.mod(which_zh,5)==0: print(f'currently working on zlevel: {which_zh}')
            flag_plane=parcel_flag[which_zh]
            cloudy_bfs=shape_search_2d(flag_plane)
            (w_values, qv_values, qc_values, th_values),zlevels=all_clouds_single_layer(t,which_zh,cloudy_bfs,vars) #outputs list of cloud averages
        return (globals()[f"{variable}_values"] for variable in vars), zlevels
            
    vars=['w','qv','qc','th']
    for variable in vars:
        globals()[f"{variable}_values"] = []
        zlevels=[]
    for t in np.arange(len(data['time'])):
    # for t in [33,34,35]: #TESTING
        if np.mod(t,1)==0: print(f'current time: {t}')
        (w_values, qv_values, qc_values, th_values),zlevels=average_clouds(t,vars) #already includes all variables


    output_file=dir+f'tracking_algorithms/plots/h5/cloud_BFS_{type}_{job_id}_wthresh{w_thresh}.h5'
    with h5py.File(output_file, 'w') as hdf: 
        hdf.create_dataset('w_values', data=w_values)
        hdf.create_dataset('qv_values', data=qv_values)
        hdf.create_dataset('qc_values', data=qc_values)
        hdf.create_dataset('th_values', data=th_values)
        hdf.create_dataset('zlevels', data=zlevels)
print('done')

In [ ]:
########################
#DEEP PROFILES

In [ ]:
#import parcel_flag_array
w_thresh=1
import h5py
# input_file = dir+'tracking_algorithms/plots/parcel_flag_array.h5' 
# with h5py.File(input_file, 'r') as f:
#     parcel_flag_array = np.array(f['flag_array'])

#import all, cloudy, updraft, or updraft_cloudy flag_array
input_file = dir+f'tracking_algorithms/plots/DEEP_all_flag_array.h5' 
with h5py.File(input_file, 'r') as f:
    deep_all_parcel_flag_array = np.array(f['flag_array'])
    
input_file = dir+f'tracking_algorithms/plots/DEEP_cloudy_flag_array_wthresh{w_thresh}.h5' 
with h5py.File(input_file, 'r') as f:
    cloudy_flag_array = np.array(f['flag_array'])

input_file = dir+f'tracking_algorithms/plots/DEEP_updraft_flag_array_wthresh{w_thresh}.h5' 
with h5py.File(input_file, 'r') as f:
    updraft_flag_array = np.array(f['flag_array'])

input_file = dir+f'tracking_algorithms/plots/DEEP_cloudyupdraft_flag_array_wthresh{w_thresh}.h5' 
with h5py.File(input_file, 'r') as f:
    cloudyupdraft_flag_array = np.array(f['flag_array'])

In [ ]:
#Collected data of averaged BFS clouds for all clouds, updrafts, and cloudy updrafts (need to change the code up to make it a 2d histogram)

#DEEP PARCEL TESTING

types=['all','cloudy','updraft','cloudyupdraft']

for type in types:
    print(f'current type: {type}')

    if type=='all': #NEED THRESHOLDED DEEP FLAG ARRAY
        flag_array=deep_all_parcel_flag_array.copy()
    if type=='cloudy': #NEED THRESHOLDED DEEP FLAG ARRAY
        flag_array=cloudy_flag_array.copy()
    elif type=='updraft':
        flag_array=updraft_flag_array.copy()
    elif type=='cloudyupdraft':
        flag_array=cloudyupdraft_flag_array.copy()
    
    def all_clouds_single_layer(t,which_zh,cloudy_bfs,vars):
        #appends the means of each individual cloud into a list
        max_ind=int(np.nanmax(cloudy_bfs))
        for ind in np.arange(1,max_ind+1):
            position=np.where(cloudy_bfs==ind)
    
            if np.any(position)==True:
                for variable in vars:
                    if variable=='w':
                        var_cloud_data=data[variable].isel(time=t).interp(zf=data['zh']).isel(zh=which_zh)[position].values
                    else:
                        var_cloud_data=data[variable].isel(time=t).isel(zh=which_zh)[position].values
                    if variable=='qv' or variable=='qc':
                        var_cloud_data*=1000
                        
                    var_cloud_data=np.mean(var_cloud_data)
                    globals()[f"{variable}_values"].append(var_cloud_data)
                    if variable=='w': #only perform once
                        zlevels.append(which_zh)
        return (globals()[f"{variable}_values"] for variable in vars), zlevels
    
    def average_clouds(t,vars):
        global zlevels
        parcel_flag=flag_array[t]
        for which_zh in np.arange(len(data['zh'])):
            if np.mod(which_zh,5)==0: print(f'currently working on zlevel: {which_zh}')
            flag_plane=parcel_flag[which_zh]
            cloudy_bfs=shape_search_2d(flag_plane)
            (w_values, qv_values, qc_values, th_values),zlevels=all_clouds_single_layer(t,which_zh,cloudy_bfs,vars) #outputs list of cloud averages
        return (globals()[f"{variable}_values"] for variable in vars), zlevels
            
    vars=['w','qv','qc','th']
    for variable in vars:
        globals()[f"{variable}_values"] = []
        zlevels=[]
    for t in np.arange(len(data['time'])):
    # for t in [33,34,35]: #TESTING
        if np.mod(t,1)==0: print(f'current time: {t}')
        (w_values, qv_values, qc_values, th_values),zlevels=average_clouds(t,vars) #already includes all variables


    if type=='all':
        output_file=dir+f'tracking_algorithms/plots/h5/DEEP_cloud_BFS_{type}.h5'
    else:
        output_file=dir+f'tracking_algorithms/plots/h5/DEEP_cloud_BFS_{type}_wthresh{w_thresh}.h5'
    with h5py.File(output_file, 'w') as hdf: 
        hdf.create_dataset('w_values', data=w_values)
        hdf.create_dataset('qv_values', data=qv_values)
        hdf.create_dataset('qc_values', data=qc_values)
        hdf.create_dataset('th_values', data=th_values)
        hdf.create_dataset('zlevels', data=zlevels)
print('done')

In [ ]:
########################################################
#Plotting

In [ ]:
w_thresh=1

#reading data back in 
def readdata(type,job_id):
    with h5py.File(dir+f'tracking_algorithms/plots/h5/cloud_BFS_{type}_{job_id}_wthresh{w_thresh}.h5', 'r') as hdf:
    # with h5py.File(dir+f'tracking_algorithms/plots/h5/cloud_BFS_{type}_{job_id}w>=0.h5', 'r') as hdf: #TESTING
        w_values = hdf['w_values'][:]
        qv_values = hdf['qv_values'][:]
        qc_values = hdf['qc_values'][:]
        th_values = hdf['th_values'][:]
        zlevels = hdf['zlevels'][:]
    zlevels=zlevels.astype(int)
    return w_values,qv_values,qc_values,th_values,zlevels
    

type='cloudy'
type='updraft'
type='cloudyupdraft'
num_jobs=30
job_id=1; [w_values,qv_values,qc_values,th_values,zlevels]=readdata(type,job_id)
vars = list(readdata(type, job_id))  # Store in a list

for job_id in range(2, num_jobs + 1):
    vars2 = list(readdata(type, job_id)) # Read the next job's data
    for i in range(len(vars)):
        vars[i] = np.concatenate((vars[i], vars2[i]))  # Update list elements
w_values, qv_values, qc_values, th_values, zlevels = vars
        


In [ ]:
def histogram2d_colors():
    colors_rgb = [
        [44, 87, 169],    # Blue 1
        [54, 92, 171],    # Blue 2
        [66, 97, 175],    # Blue 3
        [67, 107, 182],   # Blue 4
        [70, 117, 187],   # Blue 5
        [70, 125, 193],   # Blue 6
        [74, 140, 204],   # Blue 7
        [76, 157, 216],   # Blue 8
        [75, 178, 232],   # Blue 9
        [78, 192, 242],   # Blue 10
        [110, 201, 242],   # Blue 11
        [140, 207, 240],   # Blue 12
        [161, 219, 245],   # Blue 13
        [190, 226, 226],   # Blue 14
    
        [40, 150, 40],    # Green 1
        [60, 160, 60],    # Green 2
        [80, 180, 80],    # Green 3
        [120, 200, 120],  # Green 4
        
        # [225, 237, 178],   # Yellow 1
        [244, 239, 125],   # Yellow 2
        [247, 232, 75],   # Yellow 3
        [249, 213, 43],   # Yellow 4
        [247, 194, 14],   # Orange 1
        [245, 178, 15],   # Orange 2
        [242, 159, 19],   # Orange 4
        [238, 141, 22],   # Orange 5
        [236, 121, 26],   # Orange 6
        
        [230, 77, 34],   # Red 1
        [229, 51, 35],   # Red 2
        [228, 37, 30],   # Red 3
        [220, 29, 32],   # Red 4
        [205, 28, 32],   # Red 5
        
    ]
    colors = [[c / 255 for c in color] for color in colors_rgb]
    
    # #TESTING
    # custom_cmap = mcolors.ListedColormap(colors)
    
    # # Create an array to display the colorbar
    # data = np.linspace(0, 1, len(colors)).reshape(1, -1)
    
    # # Plotting the colorbar
    # fig, ax = plt.subplots(figsize=(10, 2))
    # cbar = ax.imshow(data, cmap=custom_cmap, aspect='auto')
    # ax.set_axis_off() 
    return colors

In [ ]:
# # #TESTING

# print(f"{len(np.where(qc_values>0.5)[0])*100/len(np.where(qc_values>-1000)[0])} %")

# # import numpy as np
# # import matplotlib.pyplot as plt
# # from matplotlib.colors import ListedColormap, BoundaryNorm

# # # Sample data for bin_array (replace with your actual data)
# # # bin_array = np.random.rand(10, 10) * 0.03  # Example data

# # # Define a custom colormap
# # colors = plt.cm.viridis(np.linspace(0, 1, 256))  # Use the 'viridis' colormap
# # colors[0] = [1, 1, 1, 1]  # Set the first color (for 0) to white
# # custom_cmap = ListedColormap(colors)

# # # Create a boundary norm
# # boundaries = np.linspace(0, 0.03, 256)  # Adjust as needed
# # norm = BoundaryNorm(boundaries, ncolors=len(boundaries)-1)

# # # Create the contour plot
# # contour = plt.contourf(bin_array, levels=boundaries, cmap=custom_cmap, norm=norm)
# # plt.colorbar(contour)

# # plt.title("Filled Contour Plot with 0 as White")
# # plt.xlabel("X-axis Label")
# # plt.ylabel("Y-axis Label")
# # plt.show()


In [ ]:
# #TESTING TESTING **********
# from matplotlib.colors import ListedColormap
# from matplotlib.colors import BoundaryNorm
# import seaborn as sns
# fig = plt.figure(figsize=(15, 10),constrained_layout=True)
# gs = gridspec.GridSpec(1, 1, figure=fig,)

# axis = fig.add_subplot(gs[0])  # Create subplot in GridSpec

# var='qv' #TESTING***

# ####################################################
# #Get Data
# units={'w':'(m/s)','qv':'(g/kg)','qc':'(g/kg)','th':'(K)'}
# values = globals()[f"{var}_values"]
# Nz = len(data['zf']) if var == 'w' else len(data['zh'])

# ####################### new
# #standardized perturbations for qv and th
# if var in ['qv','th']: #TESTING
#     means=[]
#     for zlevel in np.arange(np.nanmax(zlevels)+1):
#         zlevelval=values[np.where(zlevels==zlevel)]
#         if zlevelval.size > 0:  # Check if zlevelval is not empty
#                 zlevelmean = np.mean(zlevelval)
#                 zlevelstd = np.std(zlevelval)
#         else:
#             zlevelmean = 0  # Or handle it differently if needed
#             zlevelstd = 1

#         values[np.where(zlevels==zlevel)]-=zlevelmean
#         values[np.where(zlevels==zlevel)]/=zlevelstd
# ####################### new

# #Split data into groups
# if var=='w':
#     percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.05),1)
# if var=='qv':
#     percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
#     if type=='updraft' and w_thresh==0:
#         percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.1),2)
# if var=='qc':
#     percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
# if var=='th':
#     percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.05),1)
#     if type=='cloudy':
#         percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.02),1)
#     elif type=='updraft':
#         percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.1),1)
# # percentiles = np.percentile(values, np.arange(0, 101, 4)) #TESTING PERCENTILES

# #Group the data based on percentiles
# groups = np.digitize(values, percentiles, 'left')
# bin_array = np.zeros((Nz, len(percentiles)+1))
# for zlevel, group in zip(zlevels, groups):
#     bin_array[zlevel, group] += 1

# #Normalization
# row_sums = bin_array.sum(axis=1, keepdims=True)
# row_sums[row_sums == 0] = 1 #avoids nans
# bin_array /= row_sums

# ####################### new
# #Cumulative Sum 
# bin_array=np.cumsum(bin_array, axis=1)
# bin_array[bin_array >= 0.99] = 0 #nanning out those >0.95
# ####################### 

# #Plotting and Colorbar
# ########################################################################################
# ####################### 
# #custom discretized colorbar
# colors=histogram2d_colors()
# custom_cmap = mcolors.ListedColormap(colors)
# #######################

# ############ new
# bin_array[bin_array==0]=-1
# custom_cmap.set_under("white")
# ############ new

# # Plotting with the custom colormap
# im = axis.imshow(bin_array, cmap=custom_cmap, aspect="auto", vmin=0, vmax=1)
# # im = axis.imshow(bin_array, cmap=custom_cmap, aspect="auto", vmin=0, vmax=0.3) #TESTING

# # Add a colorbar for reference
# cbar = plt.colorbar(im, ax=axis, orientation='vertical', fraction=0.05, pad=0.04, shrink=0.8,label='cumulative normalized count', extend="min") #new
# #add more tick labels
# num_colors=len(colors)
# cbar.set_ticks(np.linspace(0, 1, num_colors+1))  
# cbar.set_ticklabels(np.round(np.linspace(0, 1, num_colors+1), 2))

# # # continuous colorbar
# # # https://seaborn.pydata.org/tutorial/color_palettes.html
# # cmap = sns.color_palette("viridis", as_cmap=True)
# # cmap = sns.color_palette("plasma", as_cmap=True)
# # colors = cmap(np.linspace(0, 1, 256))
# # colors[0] = [1, 1, 1, 1]  # Set the first color to white
# # custom_cmap = mcolors.ListedColormap(colors)

# # #discretized colorbar 
# # num_colors = 10  
# # custom_colors = custom_cmap(np.linspace(0, 1, num_colors+1))
# # custom_cmap = mcolors.ListedColormap(custom_colors)

# # old code without count normalization
# # num_colors=int(bin_array.max())+1
# # colors = plt.cm.coolwarm(np.linspace(0, 1, num_colors))
# # colors[0] = [1, 1, 1, 1]  # Set first color to white (RGBA)
# # discrete_cmap = ListedColormap(colors)
# # _skip = 1 if num_colors < 25  else num_colors // 25
# # norm = BoundaryNorm(np.arange(-_skip/2, num_colors - _skip/2, _skip), num_colors)
# # im = axis.imshow(bin_array, cmap=discrete_cmap, norm=norm, aspect="auto")
# # unique_counts = np.arange(0, bin_array.max()-_skip/2,_skip).astype(int)
# # cbar.set_ticks(unique_counts)  # Ensure ticks are at integer values
# # cbar.set_ticklabels(unique_counts.astype(int))
# # cbar.minorticks_off()
# ########################################################################################

# ########################################################################################
# # Add labels and title
# axis.set_title(f"2D Histogram for {var}")
# axis.set_xlabel(f"{var} {units[var]}")
# if var in ['qv','th']:
#     axis.set_title(f"2D Histogram for {var}'_norm")
#     axis.set_xlabel(f"{var}'_norm {units[var]}")
# axis.set_ylabel('z (km)')

# # Set y-ticks and labels properly
# axis.set_yticks(np.arange(Nz))
# new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# axis.set_yticklabels(new_ytick_labels, fontsize=9, rotation=0)

# # Set x-ticks
# if var=='w':
#     tick_indices = np.arange(0, len(percentiles), 20)  # Adjust step as needed
# if var=='qv':
#     tick_indices = np.arange(0, len(percentiles), 30) 
#     if type=='cloudyupdraft' and w_thresh==0:
#         tick_indices = np.arange(0, len(percentiles), 45) 
#     if type=='updraft' and w_thresh==0:
#         tick_indices = np.arange(0, len(percentiles), 20) 
# if var=='qc':
#     tick_indices = np.arange(0, len(percentiles), 10) 
# if var=='th':
#     tick_indices = np.arange(0, len(percentiles), 10) 
#     if w_thresh==0:
#         tick_indices = np.arange(0, len(percentiles), 15) 
#     if type=='updraft' and w_thresh==0:
#         tick_indices = np.arange(0, len(percentiles), 25) 
# axis.set_xticks(tick_indices)
# axis.set_xticklabels(percentiles[tick_indices], fontsize=8, rotation=0)


# # Add a grid
# # axis.xaxis.set_major_locator(plt.MultipleLocator(1))  # Show grid for every tick
# axis.grid(visible=True, axis='x', color='black', linestyle='--', linewidth=0.5)
# axis.grid(visible=True, axis='y', color='black', linestyle='--', linewidth=0.5)
# axis.invert_yaxis()


# # old set x-ticks
# # axis.set_xticks(np.arange(len(percentiles)))
# # axis.set_xticklabels(np.arange(len(percentiles)), fontsize=9, rotation=0)
# # current_xticks = axis.get_xticks()
# # new_xtick_labels = rounded_percentiles
# # axis.set_xticklabels(new_xtick_labels, fontsize=9, rotation=0)


# # #secondary x axis for values at each percentile group
# # axis2=axis.secondary_xaxis(location='top')
# # axis2.set_xticks(np.arange(len(percentiles)))
# # axis2.set_xticklabels(np.arange(len(percentiles)), fontsize=9, rotation=0)
# # current_xticks = axis2.get_xticks()
# # new_xtick_labels = rounded_percentiles
# # axis2.set_xticklabels(new_xtick_labels, fontsize=9, rotation=0)
# ########################################################################################



# # ########################################################################################
# # # Add labels and title
# # axis.set_title(f'2D Histogram for {var}')
# # axis.set_xlabel(f'Percentile Group of {var} (%)')
# # axis.set_ylabel('z (km)')

# # # Set x-ticks
# # axis.set_xticks(np.arange(len(percentiles)))
# # axis.set_xticklabels(np.arange(len(percentiles)), fontsize=9, rotation=0)
# # current_xticks = axis.get_xticks()
# # new_xtick_labels = current_xticks * 4
# # axis.set_xticklabels(new_xtick_labels, fontsize=9, rotation=0)

# # # Set y-ticks and labels properly
# # axis.set_yticks(np.arange(Nz))
# # new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
# # axis.set_yticklabels(new_ytick_labels, fontsize=9, rotation=0)

# # #secondary x axis for values at each percentile group
# # axis2=axis.secondary_xaxis(location='top')
# # axis2.set_xticks(np.arange(len(percentiles)))
# # axis2.set_xticklabels(np.arange(len(percentiles)), fontsize=9, rotation=0)
# # current_xticks = axis2.get_xticks()
# # new_xtick_labels = rounded_percentiles
# # axis2.set_xticklabels(new_xtick_labels, fontsize=9, rotation=0)

# # # Add a grid
# # axis.grid(visible=True, axis='x', color='black', linestyle='--', linewidth=0.5)
# # axis.grid(visible=True, axis='y', color='black', linestyle='--', linewidth=0.5)
# # axis.invert_yaxis()
# # ########################################################################################

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

def twod_histogram(var, axis):
    #Get Data
    units={'w':'(m/s)','qv':'(g/kg)','qc':'(g/kg)','th':'(K)'}
    values = globals()[f"{var}_values"]
    Nz = len(data['zf']) if var == 'w' else len(data['zh'])
    
    ####################### new
    #standardized perturbations for qv and th
    if var in ['qv','th']: #TESTING
        means=[]
        for zlevel in np.arange(np.nanmax(zlevels)+1):
            zlevelval=values[np.where(zlevels==zlevel)]
            if zlevelval.size > 0:  # Check if zlevelval is not empty
                    zlevelmean = np.mean(zlevelval)
                    zlevelstd = np.std(zlevelval)
            else:
                zlevelmean = 0  # Or handle it differently if needed
                zlevelstd = 1
    
            values[np.where(zlevels==zlevel)]-=zlevelmean
            values[np.where(zlevels==zlevel)]/=zlevelstd
    ####################### new
    
    #Split data into groups
    if var=='w':
        percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.05),1)
    if var=='qv':
        percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
        if type=='updraft' and w_thresh==0:
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.1),2)
    if var=='qc':
        percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
    if var=='th':
        percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.05),1)
        if type=='cloudy':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.02),1)
        elif type=='updraft':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.1),1)
    # percentiles = np.percentile(values, np.arange(0, 101, 4)) #TESTING PERCENTILES
    
    #Group the data based on percentiles
    groups = np.digitize(values, percentiles, 'left')
    bin_array = np.zeros((Nz, len(percentiles)+1))
    for zlevel, group in zip(zlevels, groups):
        bin_array[zlevel, group] += 1
    
    #Normalization
    row_sums = bin_array.sum(axis=1, keepdims=True)
    row_sums[row_sums == 0] = 1 #avoids nans
    bin_array /= row_sums
    
    ####################### new
    #Cumulative Sum 
    bin_array=np.cumsum(bin_array, axis=1)
    bin_array[bin_array >= 0.99] = 0 #nanning out those >0.95
    ####################### 

    #Plotting and Colorbar
    ########################################################################################
    ####################### new
    #custom discretized colorbar
    colors=histogram2d_colors()
    custom_cmap = mcolors.ListedColormap(colors)
    #######################

    ############ new
    bin_array[bin_array==0]=-1
    custom_cmap.set_under("white")
    ############ new
    
    # Plotting with the custom colormap
    im = axis.imshow(bin_array, cmap=custom_cmap, aspect="auto", vmin=0, vmax=1)
    
    # Add a colorbar for reference
    cbar = plt.colorbar(im, ax=axis, orientation='vertical', fraction=0.05, pad=0.04, shrink=0.8,label='cumulative normalized count', extend="min") #new
    #add more tick labels
    num_colors=len(colors)
    cbar.set_ticks(np.linspace(0, 1, num_colors+1))  
    cbar.set_ticklabels(np.round(np.linspace(0, 1, num_colors+1), 2))
    ########################################################################################

    
    #Labels and Ticks
    ########################################################################################
    
    # Add labels and title
    axis.set_title(f"2D Histogram for {var}")
    axis.set_xlabel(f"{var} {units[var]}")
    if var in ['qv','th']:
        axis.set_title(f"2D Histogram for {var}'_norm")
        axis.set_xlabel(f"{var}'_norm {units[var]}")
    if var in ['qc']:
        axis.set_title(f"2D Histogram for qc+qi'")
        axis.set_xlabel(f"qc+qi {units[var]}")
    axis.set_ylabel('z (km)')

    
    # Set y-ticks and labels properly
    axis.set_yticks(np.arange(Nz))
    new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
    axis.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
    
    # Set x-ticks
    if var=='w':
        tick_indices = np.arange(0, len(percentiles), 20)  # Adjust step as needed
    if var=='qv':
        tick_indices = np.arange(0, len(percentiles), 30) 
        if type=='cloudyupdraft' and w_thresh==0:
            tick_indices = np.arange(0, len(percentiles), 45) 
        if type=='updraft' and w_thresh==0:
            tick_indices = np.arange(0, len(percentiles), 20) 
    if var=='qc':
        tick_indices = np.arange(0, len(percentiles), 10) 
    if var=='th':
        tick_indices = np.arange(0, len(percentiles), 10) 
        if w_thresh==0:
            tick_indices = np.arange(0, len(percentiles), 15) 
        if type=='updraft' and w_thresh==0:
            tick_indices = np.arange(0, len(percentiles), 25) 
    axis.set_xticks(tick_indices)
    axis.set_xticklabels(percentiles[tick_indices], fontsize=6, rotation=0)
    
    # Add a grid
    # axis.xaxis.set_major_locator(plt.MultipleLocator(1))  # Show grid for every tick
    axis.grid(visible=True, axis='x', color='black', linestyle='--', linewidth=0.5)
    axis.grid(visible=True, axis='y', color='black', linestyle='--', linewidth=0.5)
    axis.invert_yaxis()
    

    ########################################################################################

#Running
####################################################################################
fig = plt.figure(figsize=(30, 20),constrained_layout=True)
gs = gridspec.GridSpec(2, 2, figure=fig,)

if type=='cloudy':
    fig.suptitle(f'2d Histogram of Averaged BFS Clouds (qc+qi>1e-6)')
if type=='updraft':
    fig.suptitle(f'2d Histogram of Averaged BFS Updrafts (w>{w_thresh})')
if type=='cloudyupdraft':
    fig.suptitle(f'2d Histogram of Averaged BFS Cloudy Updrafts (w>{w_thresh}) & (qc+qi>1e-6)')

vars = ['w', 'qv', 'qc', 'th']
# vars= ['w'] #TESTING
for i, var in enumerate(vars):
    ax = fig.add_subplot(gs[i])  # Create subplot in GridSpec
    twod_histogram(var, ax)







#saving plot #uncomment later
if type=='cloudy':
    plt.savefig(dir+f'tracking_algorithms/plots/'+f'cloudy_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)
elif type=='updraft':
    plt.savefig(dir+f'tracking_algorithms/plots/'+f'updraft_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)
elif type=='cloudyupdraft':
    plt.savefig(dir+f'tracking_algorithms/plots/'+f'cloudyupdraft_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap

w_thresh=1

for type in ['all','cloudy','updraft','cloudyupdraft']:
    
    input_file=dir+f'tracking_algorithms/plots/h5/DEEP_cloud_BFS_{type}_wthresh{w_thresh}.h5'
    if type=='all':
        input_file=dir+f'tracking_algorithms/plots/h5/DEEP_cloud_BFS_{type}.h5'
    with h5py.File(input_file, 'r') as hdf: #TESTING
            w_values = hdf['w_values'][:]
            qv_values = hdf['qv_values'][:]
            qc_values = hdf['qc_values'][:]
            th_values = hdf['th_values'][:]
            zlevels = hdf['zlevels'][:]
            zlevels=zlevels.astype(int)
    
    def twod_histogram(var, axis):
        #Get Data
        units={'w':'(m/s)','qv':'(g/kg)','qc':'(g/kg)','th':'(K)'}
        values = globals()[f"{var}_values"]
        Nz = len(data['zf']) if var == 'w' else len(data['zh'])
        
        ####################### new
        #standardized perturbations for qv and th
        if var in ['qv','th']: #TESTING
            means=[]
            for zlevel in np.arange(np.nanmax(zlevels)+1):
                zlevelval=values[np.where(zlevels==zlevel)]
                if zlevelval.size > 0:  # Check if zlevelval is not empty
                    zlevelmean = np.mean(zlevelval)
                    zlevelstd = np.std(zlevelval)
                else:
                    zlevelmean = 0  # Or handle it differently if needed
                    zlevelstd = 1
                
                values[np.where(zlevels==zlevel)]-=zlevelmean
                # if np.isnan(values[np.where(zlevels==zlevel)]).item()!=True:
                values[np.where(zlevels==zlevel)]/=zlevelstd

                    
        ####################### new
        
        #Split data into groups
        if var=='w':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.05),1)
        if var=='qv':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
        if var=='qc':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),2)
        if var=='th':
            percentiles=np.round(np.arange(np.nanmin(values),np.nanmax(values),0.01),1)
        # percentiles = np.percentile(values, np.arange(0, 101, 4)) #TESTING PERCENTILES
        
        #Group the data based on percentiles
        groups = np.digitize(values, percentiles, 'left')
        bin_array = np.zeros((Nz, len(percentiles)+1))
        for zlevel, group in zip(zlevels, groups):
            bin_array[zlevel, group] += 1
        
        #Normalization
        row_sums = bin_array.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1 #avoids nans
        bin_array /= row_sums
        
        ####################### new
        #Cumulative Sum 
        bin_array=np.cumsum(bin_array, axis=1)
        bin_array[bin_array >= 0.99] = 0 #nanning out those >0.95
        ####################### 
    
        #Plotting and Colorbar
        ########################################################################################
        ####################### new
        #custom discretized colorbar
        colors=histogram2d_colors()
        custom_cmap = mcolors.ListedColormap(colors)
        #######################
    
        ############ new
        bin_array[bin_array==0]=-1
        custom_cmap.set_under("white")
        ############ new
        
        # Plotting with the custom colormap
        im = axis.imshow(bin_array, cmap=custom_cmap, aspect="auto", vmin=0, vmax=1)
        # im = axis.imshow(bin_array, cmap=custom_cmap, aspect="auto", vmin=0, vmax=0.3) #TESTING
        
        # Add a colorbar for reference
        cbar = plt.colorbar(im, ax=axis, orientation='vertical', fraction=0.05, pad=0.04, shrink=0.8,label='cumulative normalized count', extend="min") #new
        #add more tick labels
        num_colors=len(colors)
        cbar.set_ticks(np.linspace(0, 1, num_colors+1))  
        cbar.set_ticklabels(np.round(np.linspace(0, 1, num_colors+1), 2))
        ########################################################################################
    
        
        #Labels and Ticks
        ########################################################################################
        
        # Add labels and title
        axis.set_title(f"2D Histogram for {var}")
        axis.set_xlabel(f"{var} {units[var]}")
        if var in ['qv','th']:
            axis.set_title(f"2D Histogram for {var}'_norm")
            axis.set_xlabel(f"{var}'_norm {units[var]}")
        axis.set_ylabel('z (km)')
    
        
        # Set y-ticks and labels properly
        axis.set_yticks(np.arange(Nz))
        new_ytick_labels = np.round(data['zf'].values[:Nz], 2)
        axis.set_yticklabels(new_ytick_labels, fontsize=8, rotation=0)
        
        # Set x-ticks
        if var=='w':
            tick_indices = np.arange(0, len(percentiles), 20)  # Adjust step as needed
        if var=='qv':
            tick_indices = np.arange(0, len(percentiles), 15) 
            if type=='updraft':
                tick_indices = np.arange(0, len(percentiles), 20) 
        if var=='qc':
            tick_indices = np.arange(0, len(percentiles), 10) 
        if var=='th':
            tick_indices = np.arange(0, len(percentiles), 10) 
        axis.set_xticks(tick_indices)
        axis.set_xticklabels(percentiles[tick_indices], fontsize=6, rotation=0)
        
        # Add a grid
        # axis.xaxis.set_major_locator(plt.MultipleLocator(1))  # Show grid for every tick
        axis.grid(visible=True, axis='x', color='black', linestyle='--', linewidth=0.5)
        axis.grid(visible=True, axis='y', color='black', linestyle='--', linewidth=0.5)
        axis.invert_yaxis()
        
    
        ########################################################################################
    
    #Running
    ####################################################################################
    fig = plt.figure(figsize=(30, 20),constrained_layout=True)
    gs = gridspec.GridSpec(2, 2, figure=fig,)
    
    if type=='all':
        fig.suptitle(f'DEEP 2d Histogram of Averaged BFS Clouds (all)')
    if type=='cloudy':
        fig.suptitle(f'DEEP 2d Histogram of Averaged BFS Clouds (qc>1e-6)')
    if type=='updraft':
        fig.suptitle(f'DEEP 2d Histogram of Averaged BFS Updrafts (w>{w_thresh})')
    if type=='cloudyupdraft':
        fig.suptitle(f'DEEP 2d Histogram of Averaged BFS Cloudy Updrafts (w>{w_thresh}) & (qc+qi>1e-6)')
    
    vars = ['w', 'qv', 'qc', 'th']
    # vars= ['w'] #TESTING
    for i, var in enumerate(vars):
        ax = fig.add_subplot(gs[i])  # Create subplot in GridSpec
        twod_histogram(var, ax)
    
    #saving plot #uncomment later
    if type=='all':
        plt.savefig(dir+f'tracking_algorithms/plots/'+f'DEEP_all_2D_histogram.jpg', dpi=300)
    if type=='cloudy':
        plt.savefig(dir+f'tracking_algorithms/plots/'+f'DEEP_cloudy_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)
    elif type=='updraft':
        plt.savefig(dir+f'tracking_algorithms/plots/'+f'DEEP_updraft_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)
    elif type=='cloudyupdraft':
        plt.savefig(dir+'tracking_algorithms/plots/'+f'DEEP_cloudyupdraft_2D_histogram_wthresh{w_thresh}.jpg', dpi=300)


In [ ]:
#################################################
#3D BFS Function

In [ ]:
def shape_search_3d(array): #3-D Shape Searching Algorithm (based on breadth-first search) 
    final_array=np.zeros_like(array); total_1s = len(np.where(array==1)[0]) 
    #Functions 
    def next_most_1_3d(array): #finds next most 1
        if len(np.where(array==1)[0])!=0:
            result=tuple(arr[0] for arr in np.where(array==1))
            return result
        else:
            return (-1,-1,-1)
    ######################################################################   
    def find_neighbor_index_3d(array,container): #find the indexes of all neighbors
                    
        #neighbors that are 0 or exist in visit or queue are not appended to neighbors
        tup,neighbors=[],[]
        
        container=[tuple(sub) for sub in container]; 
        nx,ny,nz=array.shape[2]-1,array.shape[1]-1,array.shape[0]-1
        
        for index in container:
            if index==(-1,-1,-1):
                pass
            else:
                ######################################################################  
                #code for inside #all possible neighbors
                tup=[[index[0]-1,index[1]-1,index[2]-1],[index[0]-1,index[1]-1,index[2]],[index[0]-1,index[1]-1,index[2]+1], 
                     [index[0]-1,index[1],index[2]-1],[index[0]-1,index[1],index[2]],[index[0]-1,index[1],index[2]+1],
                     [index[0]-1,index[1]+1,index[2]-1],[index[0]-1,index[1]+1,index[2]],[index[0]-1,index[1]+1,index[2]+1],

                     [index[0],index[1]-1,index[2]-1],[index[0],index[1]-1,index[2]],[index[0],index[1]-1,index[2]+1],
                     [index[0],index[1],index[2]-1],[index[0],index[1],index[2]],[index[0],index[1],index[2]+1],
                     [index[0],index[1]+1,index[2]-1],[index[0],index[1]+1,index[2]],[index[0],index[1]+1,index[2]+1],

                     [index[0]+1,index[1]-1,index[2]-1],[index[0]+1,index[1]-1,index[2]],[index[0]+1,index[1]-1,index[2]+1],
                     [index[0]+1,index[1],index[2]-1],[index[0]+1,index[1],index[2]],[index[0]+1,index[1],index[2]+1],
                     [index[0]+1,index[1]+1,index[2]-1],[index[0]+1,index[1]+1,index[2]],[index[0]+1,index[1]+1,index[2]+1],  
                    ]  
                #code for edges and corners 
                tup=[sublist for sublist in tup if 0 <= sublist[0] <= nz] 
                tup=[[sublist[0],np.mod(sublist[1],ny+1),sublist[2]] for sublist in tup] #use if y boundaries are periodic
                # tup=[sublist[0],sublist[1],[np.mod(sublist[2],nx+1)] for sublist in tup] #use if x boundaries are periodic
                tup=[sublist for sublist in tup if 0 <= sublist[2] <= nx] #use if x not periodic
                if tup: [neighbors.append(sub) for sub in tup]
                ######################################################################   
            if neighbors: #only if neighbors exists
                neighbors=np.unique(neighbors,axis=0).tolist()
                neighbors=[tuple(sub) for sub in neighbors if array[tuple(sub)] != 0 and tuple(sub) not in container] 
        
        if not neighbors:
            neighbors=np.full((0,3),0,dtype=int)    
        else: neighbors=np.array(neighbors)
        return neighbors
    ###################################################################### 
    
    #The Algorithm
    n,k=0,0 #n is current shape, k is number of iterations on current shape 
    
    while True:
        n+=1;
        if np.mod(n,100)==0:
             found_1s=len(np.where(array==1)[0]); print("Current shape number: " + str(n) + " (" + str(round(100*(1-found_1s/total_1s),2)) + "%)")
        visit=np.full((0,3),0,dtype=int) #creates visit variable for current shapes
        queue=np.full((0,3),0,dtype=int) #creates queue variable for possible neighbors

        ######################################################################
        #finds the first 1 in the array, append to visit, and zero out
        index=next_most_1_3d(array); 
        visit=np.concatenate([visit,[index]]);array[index]=0;

        #finds neighbors of first index and appends to queue
        neighbors=find_neighbor_index_3d(array,visit) 
        queue=np.concatenate([queue,neighbors])
        
        #for rest of individual shape
        while queue.size!=0: 
            k+=1
            #add all queued indexes to visit and find unique,nonzero,and nonvisited neighbor
            visit=np.concatenate([visit,queue]);neighbors=find_neighbor_index_3d(array,queue) 
            #zero out queued in array and empty out queue
            array[queue[:,0],queue[:,1],queue[:,2]]=0; queue=np.full((0,3),0,dtype=int) 
            #append found neighbor to queue
            queue=np.concatenate([queue,neighbors]) #append additional neighbors to queue
            
            #failsafes to end while loop    
            if queue.size==0: 
                break #breaks single loop for current shape if no more neighbor
            if k>=array.shape[0]*array.shape[1]*array.shape[2]: #k failsafe, set to max volume of array
                break   
        #sets output array's value to shape number
        if index!=(-1,-1,-1):
            final_array[visit[:,0],visit[:,1],visit[:,2]] = n #sets final array location to shape number
        ######################################################################  
        
        #failsafes to end while loop    
        if index==(-1,-1,-1):
            print("no more shapes are left")
            print(f"total of {n-1} shapes were found")
            break  #break loop if no more 1s
        if n>=array.shape[0]*array.shape[1]*array.shape[2]:  #n failsafe, set to max volume of array
            print("failsafe reached")
            break  #break loop if no more 1s
    #Save final array to external nc file
    xr.DataArray(final_array, dims=('z','y', 'x'),name='var').to_netcdf('cloud_shapes_3d.nc')
    return final_array #15s/1t==>35m/140t
##########################################################################################################################################  

In [ ]:
def binned_scatter3d(values,zh,axis):
    num_groups = 10
    min_val, max_val = np.nanmin(values), np.nanmax(values)
    bins = np.linspace(min_val, max_val, num_groups + 1)
    group_indices = np.digitize(values, bins, right=False) - 1  # subtract 1 to make it zero-indexed
    # colors = cm.seismic(np.linspace(0, 1, num_groups))
    colors = cm.Spectral(np.linspace(0, 1, num_groups))
    
    # ####### Optional
    # original_cmap = cm.Spectral
    
    # # Function to darken higher values
    # def darken_colormap(cmap, factor):
    #     # Extract the RGB values from the colormap
    #     colors = cmap(np.linspace(0, 1, 256))
    #     # Darken the higher colors by scaling the RGB values
    #     colors[:, :3] *= np.linspace(1, 1-factor, 256)[:, np.newaxis]
    #     # Ensure alpha values are set to 1 (fully opaque)
    #     colors[:, -1] = 1.0
    #     # Create a new colormap from the modified colors
    #     return mcolors.ListedColormap(colors)
    
    # # Create a darkened colormap
    # darkened_cmap = darken_colormap(original_cmap, factor=0.2) #what percent to darken the colors
    
    # # Generate colors for the number of groups
    # colors = darkened_cmap(np.linspace(0, 1, num_groups))
    # ####### 
    
    for i in range(num_groups):
        mask = group_indices == i
        # axis.scatter(np.array(values)[mask], zh[mask], color=colors[i],alpha=1,label=f'> {bins[i]:.2f}')
        axis.scatter(np.array(values)[mask], np.array(zh)[mask], color=colors[i],alpha=1,label=f'> {bins[i]:.2f}')
    axis.legend(loc='center left', bbox_to_anchor=(1, 0.5))

In [ ]:
def all_clouds_single_layer(t,cloudy_bfs,variable):
    #appends the means of each individual cloud into a list
    max_ind=int(np.nanmax(cloudy_bfs))
    for ind in np.arange(1,max_ind+1):
        if np.mod(ind,100)==0: print(f'currently working cloud # {ind}/{max_ind}')
        position=np.where(cloudy_bfs==ind)
    
        if np.any(position)==True:
            if variable=='w':
                var_cloud_data=data[variable].isel(time=t).interp(zf=data['zh'])[position].values
            else:
                var_cloud_data=data[variable].isel(time=t)[position].values
            if variable=='qv' or variable=='qc':
                var_cloud_data*=1000
            var_cloud_data=np.mean(var_cloud_data)
            clouds.append(var_cloud_data)

            CTH=np.where(cloudy_bfs==ind)[0].max() #Cloud Top Height
            CBH=np.where(cloudy_bfs==ind)[0].min()
            # CH=CTH-CBH #Cloud Height
            CH=data['zf'].values[CTH].item()-data['zf'].values[CBH].item() #Cloud Height
            zlevs.append(CH)

def average_clouds(t,variable):
    global clouds,zlevs,cloudy_bfs
    clouds,zlevs=[],[]
    parcel_flag=cloudy_flag_array[t]
    flag_plane=parcel_flag.copy()
    if variable=='w':
        cloudy_bfs=shape_search_3d(flag_plane) #only runs once the first time and then saved as global
    all_clouds_single_layer(t,cloudy_bfs,variable)
    return clouds,zlevs

start_time = time.time();
##############################

# t=33
# [clouds,CHs]=average_clouds(t,'w') #60s/1t==>140m/140t

# # xlabel='w (m/s)'
# fig, axs = plt.subplots(1, 1, figsize=(5, 5))
# binned_scatter3d(clouds,CHs,axs)
# fig.suptitle(f"averaged cloudy parcel 3d-groups data by height")
# plt.ylabel('Cloud Height Grid Box');plt.xlabel(xlabel); ###  ***

# ##############################
# end_time = time.time(); elapsed_time = end_time - start_time
# print(f"Total Elapsed Time: {elapsed_time} seconds")
# print('done')


#Plotting
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
fig.suptitle(f"averaged deep convective cloudy parcel groups data by height")
ax1, ax2, ax3, ax4 = axs.flatten()
plt.tight_layout()

start_time = time.time();
##############################
vars=['w','qv','qc','th'] 
xlabels=['w (m/s)','qv (g/kg)','qc (g/kg)','th (K)']
axises=[ax1,ax2,ax3,ax4]
# for t in np.arange(len(data['time'])):
# for t in [11,12,13]: #TESTING
for t in [33]: #TESTING
    for variable,axis,xlabel in zip(vars,axises,xlabels):
        print(f'working on {variable}')
        [clouds,CHs]=average_clouds(t,variable) #60s/1t==>140m/140t
        fig.subplots_adjust(wspace=0.5) 
        binned_scatter3d(clouds,CHs,axis) ###***
        axis.set_ylabel('Cloud Height (km)');axis.set_xlabel(xlabel);
##############################
end_time = time.time(); elapsed_time = end_time - start_time
print(f"Total Elapsed Time: {elapsed_time} seconds")
print('done')


In [ ]:
plt.hist(data['zh'][CHs], bins=20, edgecolor='black')  # You can adjust the number of bins as needed
plt.ylabel('Counts')
plt.xlabel('Cloud Height Grid Box')
plt.title('Histogram of Cloud Heights')

In [ ]:
# #TESTING
# num_groups = 10
# min_val, max_val = np.nanmin(values), np.nanmax(values)
# bins = np.linspace(min_val, max_val, num_groups + 1)
# group_indices = np.digitize(values, bins, right=False) - 1  # subtract 1 to make it zero-indexed

In [ ]:
# # TESTING
# #making contour matrix (does not take averages only adds last of each group currently)
# ngroups=len(grouped_data)

# maxrow=-1
# for ind in np.arange(0,ngroups):
#     length=len(grouped_data[ind])
#     if length>maxrow:
#         maxrow=length
# matrix=np.zeros((maxrow,ngroups))

# for group in np.arange(ngroups):
#     current_group=grouped_data[group]
#     for ind in np.arange(len(current_group)):
#         val=current_group[ind][0]
#         row=current_group[ind][1]
#         matrix[row,group]=val


# ######

# matrix=matrix[0:4,:]
# repeat_rows = 3
# repeat_cols = 3
# matrix = np.tile(matrix, (repeat_rows, repeat_cols))
# ######
 
# matrix_with_nan = np.where(matrix == 0, np.nan, matrix)

# masked_matrix = np.ma.masked_invalid(matrix_with_nan)
# cmap = plt.get_cmap('viridis').copy()
# cmap.set_bad("white")

# fig, ax = plt.subplots(figsize=(6,4))
# # contour = ax.contourf(masked_matrix, cmap=cmap)
# contour = ax.contourf(matrix, cmap=cmap)
# cbar = fig.colorbar(contour, ax=ax)
# cbar.set_label('w (m/s)')
# ax.set_ylabel('Height (CH Grid Diff)')
# ax.set_xlabel('Group Number in Integers')
# plt.tight_layout()
# plt.show()


In [ ]:
#TESTING Cloud BFS Averaging Algorithm
# # #TESTING
# # # one=np.array(w_values.copy())
# # # two=np.array(qv_values.copy())
# # # three=np.array(zlevels.copy())
# # one2=np.array(w_values.copy())
# # two2=np.array(qv_values.copy())
# # three2=np.array(zlevels.copy())

# types=['cloudy','updraft']
# job_id=5

# def readdata(type,job_id):
#     with h5py.File(dir+f'tracking_algorithms/plots/h5/cloud_BFS_{type}_{job_id}.h5', 'r') as hdf:
#         w_values = hdf['w_values'][:]
#         qv_values = hdf['qv_values'][:]
#         qc_values = hdf['qc_values'][:]
#         th_values = hdf['th_values'][:]
#         zlevels = hdf['zlevels'][:]
#     zlevels=zlevels.astype(int)
#     return w_values,qv_values,qc_values,th_values,zlevels


# for type in types:
#     [w_values,qv_values,qc_values,th_values,zlevels]=readdata(type,job_id)
#     if type=='cloudy':
#         one=np.array(w_values.copy())
#         two=np.array(qv_values.copy())
#         three=np.array(zlevels.copy())
#     elif type=='updraft':
#         one2=np.array(w_values.copy())
#         two2=np.array(qv_values.copy())
#         three2=np.array(zlevels.copy())

# #TESTING TESTING *************
# #reading data back in 
# def readdata(type,job_id):
#     with h5py.File(dir+f'tracking_algorithms/plots/h5/cloud_BFS_{type}_{job_id}.h5', 'r') as hdf:
#         w_values = hdf['w_values'][:]
#         qv_values = hdf['qv_values'][:]
#         qc_values = hdf['qc_values'][:]
#         th_values = hdf['th_values'][:]
#         zlevels = hdf['zlevels'][:]
#     zlevels=zlevels.astype(int)
#     return w_values,qv_values,qc_values,th_values,zlevels
    

# # type='cloudy'
# # type='updraft'
# type='cloudyupdraft'
# num_jobs=30
# job_id=1; [w_values,qv_values,qc_values,th_values,zlevels]=readdata(type,job_id)
# vars = list(readdata(type, job_id))  # Store in a list

# for job_id in range(2, num_jobs + 1):
#     vars2 = list(readdata(type, job_id)) # Read the next job's data
#     for i in range(len(vars)):
#         vars[i] = np.concatenate((vars[i], vars2[i]))  # Update list elements
# w_values, qv_values, qc_values, th_values, zlevels = vars
# #************** this method has some issue


# #TESTING TESTING *************
# # type='cloudyupdraft'
# # num_jobs=30
# # job_id=4; [w_values,qv_values,qc_values,th_values,zlevels]=readdata(type,job_id)
# # vars = list(readdata(type, job_id))  # Store in a list
# # vars

# # type='updraft'
# # num_jobs=30
# # job_id=4; [w_values,qv_values,qc_values,th_values,zlevels]=readdata(type,job_id)
# # vars2 = list(readdata(type, job_id))  # Store in a list
# # vars2

# # test=vars[0]==vars2[0]
# # plt.plot(test)

# # one=w_values.copy()
# # two=w_values.copy()
# # three=w_values.copy()
# test=one==two
# plt.plot(test,color='k')
# test=one==three
# plt.plot(test+1,color='blue')
# test=two==three
# plt.plot(test+2,color='green')


In [ ]:
# percents=np.arange(0, 101, 4)

# for var in vars:
#     values = globals()[f"{var}_values"]
#     Nz = len(data['zf']) if var == 'w' else len(data['zh'])
#     globals()[f"{var}_percentile"] = np.round(np.percentile(values, np.arange(0, 101, 4)),3)

# import matplotlib.pyplot as plt
# import numpy as np

# # Ensure all arrays are of the same length
# assert len(w_percentile) == len(percents) == len(qv_percentile)== len(qc_percentile)== len(th_percentile), "Arrays must be of the same length."

# # Create a figure
# fig, ax = plt.subplots(figsize=(8, 4))

# # Create key data for the table
# key_data = np.vstack((percents, w_percentile, qv_percentile, qc_percentile, th_percentile)).T

# # Create the table
# table = ax.table(cellText=key_data, colLabels=['Percentile', 'w (m/s)', 'qv (g/kg)', 'qc (g/kg)', 'th (g/kg)'], 
#                  loc='center', cellLoc='center', colColours=['lightgrey', 'lightgrey', 'lightgrey', 'lightgrey', 'lightgrey'])
# table.auto_set_font_size(False)
# table.set_fontsize(10)
# table.scale(1, 1.5)  # Adjust table height
# ax.axis('off')


# if type=='cloudy':
#     fig.suptitle('Percentiles 2D Histogram of Averaged BFS Clouds', fontsize=14, y=1.4)
# elif type=='updraft':
#     fig.suptitle('Percentiles 2D Histogram of Averaged BFS Updrafts', fontsize=14, y=1.4)
# elif type=='cloudyupdraft':
#     fig.suptitle('Percentiles 2D Histogram of Averaged BFS Cloudy Updrafts', fontsize=14, y=1.4)

In [ ]:
#TESTING

# xlabels=['w (m/s)','qv (g/kg)','qc (g/kg)','th (K)']
# values=z
# num_groups = 25
# min_val, max_val = np.nanmin(values), np.nanmax(values)
# bins = np.linspace(min_val, max_val, num_groups + 1)
# group_indices = np.digitize(values, bins, right=False) - 1  # subtract 1 to make it zero-indexed

# from scipy.interpolate import griddata
# x=group_indices
# # Create a grid for contouring, adjusted to data ranges
# min_x, max_x = np.nanmin(x), np.nanmax(x)
# min_y, max_y = np.nanmin(y), np.nanmax(y)

# grid_x, grid_y = np.mgrid[min_x:max_x:100j, min_y:max_y:100j]  # Create a grid for the contour
# grid_z = griddata((x, y), z, (grid_x, grid_y), method='linear')  # Interpolate to grid

# # Plotting
# plt.figure(figsize=(8, 6))
# plt.contourf(grid_x, grid_y, grid_z, levels=20, cmap='viridis')  # Create contour plot
# plt.colorbar(label='w (m/s)')  # Optional: Add a colorbar
# plt.title("Interpolated Scatter Data")
# plt.xlabel("Group Number")
# plt.ylabel("Z Level")

# plt.ylim(7,26)
# plt.xticks(np.arange(min_x, max_x + 1, 1));

# # current_y_ticks = plt.gca().get_yticks();
# # current_y_tick_labels = plt.gca().get_yticklabels();
# # new_y_tick_labels = [
# #     str(round(data['zh'][int(float(label.get_text()))].values * 1, 1)) 
# #     for label in current_y_tick_labels
# # ]
# # # new_y_tick_labels = [
# # #     str(round(data['zh'][int(float(label.get_text()))].values * 1000, 1)) 
# # #     for label in current_y_tick_labels
# # # ]
# # plt.gca().set_yticks(current_y_ticks);  # Ensure tick positions match
# # plt.gca().set_yticklabels(new_y_tick_labels);


# values=z
# num_groups = 25
# min_val, max_val = np.nanmin(values), np.nanmax(values)
# bins = np.linspace(min_val, max_val, num_groups + 1)
# group_indices = np.digitize(values, bins, right=False) - 1  # subtract 1 to make it zero-indexed

# matrix=np.zeros((len(data['zh']),group_indices.max()+1))
# for (group,yind,zind) in zip(group_indices,y,z):
#     matrix[yind,group]=zind

# num_levels=20
# vmin=-5
# vmax=15
# # matrix = np.where(matrix == 0, np.nan, matrix)
# plt.contourf(matrix,levels=num_levels,cmap='viridis',vmin=vmin, vmax=vmax)

# # plt.xlim(0, 15)
# # plt.ylim(5, 25)
# plt.xlabel('group number')
# plt.ylabel('z grid')
# plt.colorbar(label='w (m/s)')

In [ ]:
# #Function scatter plot colored by bins #not needed
# def binned_scatter(values,zh,axis):
#     num_groups = 10
#     min_val, max_val = np.nanmin(values), np.nanmax(values)
#     bins = np.linspace(min_val, max_val, num_groups + 1)
#     group_indices = np.digitize(values, bins, right=False) - 1  # subtract 1 to make it zero-indexed
#     # colors = cm.seismic(np.linspace(0, 1, num_groups))
#     colors = cm.Spectral(np.linspace(0, 1, num_groups))
    
#     # ####### Optional
#     # original_cmap = cm.Spectral
    
#     # # Function to darken higher values
#     # def darken_colormap(cmap, factor):
#     #     # Extract the RGB values from the colormap
#     #     colors = cmap(np.linspace(0, 1, 256))
#     #     # Darken the higher colors by scaling the RGB values
#     #     colors[:, :3] *= np.linspace(1, 1-factor, 256)[:, np.newaxis]
#     #     # Ensure alpha values are set to 1 (fully opaque)
#     #     colors[:, -1] = 1.0
#     #     # Create a new colormap from the modified colors
#     #     return mcolors.ListedColormap(colors)
    
#     # # Create a darkened colormap
#     # darkened_cmap = darken_colormap(original_cmap, factor=0.2) #what percent to darken the colors
    
#     # # Generate colors for the number of groups
#     # colors = darkened_cmap(np.linspace(0, 1, num_groups))
#     # ####### 
#     for i in range(num_groups):
#         mask = group_indices == i
#         axis.scatter(np.array(values)[mask], zh[mask], color=colors[i],alpha=1,label=f'> {bins[i]:.2f}')
#     axis.legend(loc='center left', bbox_to_anchor=(1, 0.5))

 # fig.subplots_adjust(wspace=0.5) 
 #        binned_scatter(clouds,data['zh'][zlevs],axis)

 #        #for contour plot
 #        z.extend(clouds)
 #        y.extend(zlevs)
 #        axis.set_ylabel('z (km)');axis.set_xlabel(xlabel);
        


# #TESTING Scatter plot of averaged values for deep convective clouds

# def deep_clouds_single_layer(t,which_zh,cloudy_bfs,variable):
#     max_ind=int(np.nanmax(cloudy_bfs))
#     for ind in np.arange(1,max_ind+1):
#         position=np.where(cloudy_bfs==ind)

#         parcel_id=parcel_flag_array[t,which_zh][position] ######
#         where = np.where(np.isin(out_nz[:, 0], parcel_id))
#         if np.any(where)==True: 
#             if np.any(position)==True:
#                 if variable=='w':
#                     var_cloud_data=data[variable].isel(time=t).interp(zf=data['zh']).isel(zh=which_zh)[position].values
#                 else:
#                     var_cloud_data=data[variable].isel(time=t).isel(zh=which_zh)[position].values
#                 if variable=='qv' or variable=='qc':
#                     var_cloud_data*=1000
#                 var_cloud_data=np.mean(var_cloud_data)
#                 clouds.append(var_cloud_data)
#                 zlevs.append(which_zh)
        

# def average_clouds(t,variable):
#     global clouds,zlevs
#     clouds,zlevs=[],[]
#     parcel_flag=cloudy_flag_array[t]
#     for which_zh in np.arange(len(data['zh'])):
#         flag_plane=parcel_flag[which_zh]
#         cloudy_bfs=shape_search_2d(flag_plane)
#         deep_clouds_single_layer(t,which_zh,cloudy_bfs,variable)
    
#         # plt.contourf(cloudy_bfs) #TESTING
#         # if which_zh==10: break #TESTING***
#     return clouds,zlevs

# #Plotting
# fig, axs = plt.subplots(2, 2, figsize=(8, 8))
# fig.suptitle(f"averaged deep convective cloudy parcel groups data by height")
# ax1, ax2, ax3, ax4 = axs.flatten()
# plt.tight_layout()

# vars=['w','qv','qc','th']
# xlabels=['w (m/s)','qv (g/kg)','qc (g/kg)','th (K)']
# axises=[ax1,ax2,ax3,ax4]
# # for t in np.arange(len(data['time'])):
# # for t in [11,12,13]: #TESTING
# for t in [11]: #TESTING
#     for variable,axis,xlabel in zip(vars,axises,xlabels):
#         print(f'working on {variable}')
#         [clouds,zlevs]=average_clouds(t,variable)

#         fig.subplots_adjust(wspace=0.5) 
#         binned_scatter(clouds,data['zh'][zlevs],axis)
#         # axis.scatter(clouds,data['zh'][zlevs],color='b')
        
#         axis.set_ylabel('z (km)');axis.set_xlabel(xlabel);      

In [ ]:
# #TESTING
# #If interested in connecting lagrangian BFS with neighboring non-lagrangian clouds 

# #Import Data
# #####################################################################  
# data2=np.array([[1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
#                 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
#                 [0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,1,0,0],
#                 [0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,1,0,0],
#                 [0,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,1,1,1,0,0],
#                 [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
#                 [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1],
#                 [0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,1,1,1,0,0],
#                 [0,0,0,1,0,0,0,0,0,1,1,0,1,1,1,0,1,1,1,0,0],
#                 [0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0],
#                 [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0],
#                 [1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1]])  
# array=data2.copy() #creates temporary array variable 

# #Running the Algorithm
# ######################################################################  
# start_time = time.time(); 
# final_array = shape_search_2d(array)
# end_time = time.time(); elapsed_time = end_time - start_time
# print(f"Total Elapsed Time: {elapsed_time} seconds")
# ###################################################################### 

# #Plotting for Testing
# ######################################################################  
# plt.subplot(2, 1, 1)
# plot_array=data2.copy(); 
# plt.imshow(plot_array,cmap='viridis')
# plt.subplot(2, 1, 2)
# plt.imshow(final_array,cmap='gist_ncar')
# ######################################################################  